In [4]:
import json

# The current nutrition facts we can think of. These may be modified later.
# Remember that some of the nutrition facts can be mssing in the JSON file.
lookUpTable = ["calories", "calories_from_fat", "total_fat", "saturated_fat", "trans_fat",
              "cholesterol", "sodium", "carbohydrates", "dietary_fiber", "sugars",
              "protein"]

# Maybe we will need a DRI table in the future
# DRI = {...}

# I'll just use the DRI value of a 21 years old, 6 ft 0 inches tall, 187 lbs man with active activity level for now
# The information is gain from https://www.nal.usda.gov/human-nutrition-and-food-safety/dri-calculator/
'''
Calories: 3200
Calories from Fat: 400-1200 (depending on dietary preferences)
Total Fat: 88 grams
Saturated Fat: 22 grams
Trans Fat: 0 grams
Cholesterol: 300 mg
Sodium: 2400-3600 mg (depending on sweat rate and dietary preferences)
Carbohydrates: 375 grams
Dietary Fiber: 38 grams
Sugars: 100 grams (or less, depending on dietary preferences)
Protein: 150 grams
'''
rdi = {"calories": 3200, "calories_from_fat": 500, "total_fat": 88, "saturated_fat": 22,
               "trans_fat": 0, "cholesterol": 300, "sodium": 2500, "carbohydrates": 375,
               "dietary_fiber": 38, "sugars": 100, "protein": 150}

In [5]:
class RecommendationSystem:
    def __init__(self):
        self.pref = {}
        self.items = {}
    
    # I copied https://stackoverflow.com/questions/51362252/javascript-cosine-similarity-function
    def cosineSimilarity(self, A, B):
        dotproduct = 0
        mA = 0
        mB = 0
        for i in range(len(A)):
            dotproduct += (A[i] * B[i])
            mA += (A[i] * A[i])
            mB += (B[i] * B[i])
        mA = sqrt(mA)
        mB = sqrt(mB)
        similarity = (dotproduct) / (mA * mB)
        return similarity
    
    def transform(self, nutritionFact, amount):
        
        # Calculate the amount of the nutrient as a percentage of RDI
        nutrient_percentage = (amount / rdi[nutritionFact]) * 100
        
        # Map the percentage to a value between 0 and 10 using a linear scale
        value = (nutrient_percentage / 10)
        value = min(value, 10)  # Cap the value at 10
        value = max(value, 0)   # Cap the value at 0
        
        return value
    
    def cmp(self, item):
        A = []
        B = []
            
        for nutritionFact, value in self.pref.items():
        
            '''
            We only consider the food items that have specified every nutrition facts specified by the user.
            The user who wants low calories food may be disappointed in some extremely high calories food that was
            forgotten to be tagged "high calories" by the restaurant, even though the other nutrition facts match perfectly. 
            '''
            if (not(nutritionFact in item['nutrition_facts'])):
                return 1e9
            A.append(value)
            
            # we need to change the amount to a value from 0 ~ 10
            B.append(transform(nutritionFact, item['nutrition_facts'][nutritionFact]))
    
        return self.cosineSimilarity(A, B)
    
    # input: A JSON file containing the restaurants "restaurantsFile"; a JSON file containing the users "usersFile"; number K
    # output: top K items based on the user's preferences.
    def recommend (self, restaurantsFile, usersFile, K):
    
        # file -> string
        restaurantsString = restaurantsFile.read()
        usersString = usersFile.read()
    
        # parse strings
        restaurants = json.loads(restaurantsString)
        users = json.loads(usersString)
    
        # Refer to Delphi/NOSQL/restaurants.json and Delphi/NOSQL/users.json
        self.items = restaurants['menu_items'];
        self.pref = users['preferences'];
        
        # for testing
        print(self.pref)
    
        # Sort the items based on the user preferences
        sortedItems = sorted(self.items, key = self.cmp)
    
        return sortedItems[0: K]
    

In [ ]:
restaurantsFile = open("restaurants.txt")
usersFile = open("users.txt")
rec = RecommendationSystem()
print (rec.recommend(restaurantsFile, usersFile, 10))